In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from imblearn.under_sampling import RandomUnderSampler
import optuna

# 데이터 로드
train = pd.read_csv("../data/train.csv").drop(columns=['ID'])
test = pd.read_csv("../data/test.csv").drop(columns=['ID'])

X = train.drop('임신 성공 여부', axis=1)
y = train['임신 성공 여부']

categorical_columns = [
    "시술 시기 코드", "시술 당시 나이", "시술 유형", "특정 시술 유형", "배란 자극 여부", "배란 유도 유형",
    "단일 배아 이식 여부", "착상 전 유전 검사 사용 여부", "착상 전 유전 진단 사용 여부", "남성 주 불임 원인",
    "남성 부 불임 원인", "여성 주 불임 원인", "여성 부 불임 원인", "부부 주 불임 원인", "부부 부 불임 원인",
    "불명확 불임 원인", "불임 원인 - 난관 질환", "불임 원인 - 남성 요인", "불임 원인 - 배란 장애",
    "불임 원인 - 여성 요인", "불임 원인 - 자궁경부 문제", "불임 원인 - 자궁내막증", "불임 원인 - 정자 농도",
    "불임 원인 - 정자 면역학적 요인", "불임 원인 - 정자 운동성", "불임 원인 - 정자 형태", "배아 생성 주요 이유",
    "총 시술 횟수", "클리닉 내 총 시술 횟수", "IVF 시술 횟수", "DI 시술 횟수", "총 임신 횟수", "IVF 임신 횟수",
    "DI 임신 횟수", "총 출산 횟수", "IVF 출산 횟수", "DI 출산 횟수", "난자 출처", "정자 출처",
    "난자 기증자 나이", "정자 기증자 나이", "동결 배아 사용 여부", "신선 배아 사용 여부", "기증 배아 사용 여부",
    "대리모 여부", "PGD 시술 여부", "PGS 시술 여부"
]

# 카테고리형 컬럼을 문자열로 변환
for col in categorical_columns:
    X[col] = X[col].astype(str)
    test[col] = test[col].astype(str)

ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
X[categorical_columns] = ordinal_encoder.fit_transform(X[categorical_columns])
test[categorical_columns] = ordinal_encoder.transform(test[categorical_columns])

numeric_columns = [
    "임신 시도 또는 마지막 임신 경과 연수", "총 생성 배아 수", "미세주입된 난자 수", "미세주입에서 생성된 배아 수",
    "이식된 배아 수", "미세주입 배아 이식 수", "저장된 배아 수", "미세주입 후 저장된 배아 수", "해동된 배아 수",
    "해동 난자 수", "수집된 신선 난자 수", "저장된 신선 난자 수", "혼합된 난자 수", "파트너 정자와 혼합된 난자 수",
    "기증자 정자와 혼합된 난자 수", "난자 채취 경과일", "난자 해동 경과일", "난자 혼합 경과일", "배아 이식 경과일",
    "배아 해동 경과일"
]

X[numeric_columns] = X[numeric_columns].fillna(0)
test[numeric_columns] = test[numeric_columns].fillna(0)

# 🎯 1:1 비율로 Test 데이터 분할 (20% 중 절반이 1, 절반이 0)
test_size = int(len(y) * 0.2 / 2)

y_0 = y[y == 0]
y_1 = y[y == 1]

# 1과 0에서 동일한 개수 추출 (각각 test_size만큼)
y_test_0 = y_0.sample(n=test_size, random_state=42)
y_test_1 = y_1.sample(n=test_size, random_state=42)

# 나머지 데이터는 train으로 사용
y_train_0 = y_0.drop(y_test_0.index)
y_train_1 = y_1.drop(y_test_1.index)

# X도 동일하게 맞춰줌
X_test = pd.concat([X.loc[y_test_0.index], X.loc[y_test_1.index]])
y_test = pd.concat([y_test_0, y_test_1])

X_train = pd.concat([X.loc[y_train_0.index], X.loc[y_train_1.index]])
y_train = pd.concat([y_train_0, y_train_1])

# 🎯 언더샘플링 적용 (0과 1을 동일 개수로 맞춤)
rus = RandomUnderSampler(sampling_strategy=1.0, random_state=42)
X_train_resampled, y_train_resampled = rus.fit_resample(X_train, y_train)

In [ ]:
# Optuna 최적화 함수
def objective(trial):
    lr_C = trial.suggest_float("lr_C", 0.01, 10.0)
    dt_max_depth = trial.suggest_int("dt_max_depth", 2, 20)
    knn_neighbors = trial.suggest_int("knn_neighbors", 3, 15)
    rf_n_estimators = trial.suggest_int("rf_n_estimators", 50, 300)
    
    models = [
        ('lr', LogisticRegression(C=lr_C, random_state=42)),
        ('dt', DecisionTreeClassifier(max_depth=dt_max_depth, random_state=42)),
        ('lda', LinearDiscriminantAnalysis()),
        ('rf', RandomForestClassifier(n_estimators=rf_n_estimators, random_state=42)),
        ('knn', KNeighborsClassifier(n_neighbors=knn_neighbors))
    ]
    
    voting_type = trial.suggest_categorical("voting", ["hard", "soft"])
    model = VotingClassifier(estimators=models, voting=voting_type)
    model.fit(X_train_resampled, y_train_resampled)
    y_pred = model.predict(X_test)
    return accuracy_score(y_test, y_pred)

# Optuna 실행
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=30)

# 최적 하이퍼파라미터 가져오기
best_params = study.best_params

# 최적 하이퍼파라미터로 모델 생성
models = [
    ('lr', LogisticRegression(C=best_params['lr_C'], random_state=42)),
    ('dt', DecisionTreeClassifier(max_depth=best_params['dt_max_depth'], random_state=42)),
    ('lda', LinearDiscriminantAnalysis()),
    ('rf', RandomForestClassifier(n_estimators=best_params['rf_n_estimators'], random_state=42)),
    ('knn', KNeighborsClassifier(n_neighbors=best_params['knn_neighbors']))
]

# 하드 보팅 모델 학습 및 평가
hard_voting_model = VotingClassifier(estimators=models, voting='hard')
hard_voting_model.fit(X_train_resampled, y_train_resampled)
y_pred_hard = hard_voting_model.predict(X_test)

# 소프트 보팅 모델 학습 및 평가
soft_voting_model = VotingClassifier(estimators=models, voting='soft')
soft_voting_model.fit(X_train_resampled, y_train_resampled)
y_pred_soft = soft_voting_model.predict(X_test)

# 평가 지표 계산
hard_acc = accuracy_score(y_test, y_pred_hard)
soft_acc = accuracy_score(y_test, y_pred_soft)

hard_cm = confusion_matrix(y_test, y_pred_hard)
soft_cm = confusion_matrix(y_test, y_pred_soft)

# 결과 출력
print("Hard Voting Accuracy:", hard_acc)
print("Hard Voting Confusion Matrix:\n", hard_cm)
print("\nSoft Voting Accuracy:", soft_acc)
print("Soft Voting Confusion Matrix:\n", soft_cm)

# 최적 모델로 최종 예측 수행
y_pred_final = soft_voting_model.predict(test)

# 제출 파일 생성
sample_submission = pd.read_csv('../submission/sample_submission.csv')
sample_submission['prediction'] = y_pred_final
sample_submission.to_csv('../submission/baseline_submit2.csv', index=False)

[I 2025-02-18 00:30:29,715] A new study created in memory with name: no-name-53192310-6712-4ec0-b130-4a58f1a52f7a
C:\Users\pung\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
[I 2025-02-18 00:31:16,154] Trial 0 finished with value: 0.6663740979130096 and parameters: {'lr_C': 4.007546644678651, 'dt_max_depth': 13, 'knn_neighbors': 14, 'rf_n_estimators': 218, 'voting': 'soft'}. Best is trial 0 with value: 0.6663740979130096.
C:\Users\pung\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to conv